## Importing Libraries

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msn
import pandas as pd
import os
os.system('apt-get install p7zip')
!pip install pyunpack
!pip install patool
from pyunpack.cli import Archive
import numpy as np
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
import time
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier
from sklearn.model_selection import RandomizedSearchCV
import xgboost
import lightgbm as lgb
from sklearn import tree
import seaborn as sns
import matplotlib.pyplot as plt
import time
# Deep learning libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Input, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.metrics import roc_auc_score

In [ ]:
directory = './'
Archive('../input/kkbox-music-recommendation-challenge/train.csv.7z').extractall(directory)
Archive('../input/kkbox-music-recommendation-challenge/test.csv.7z').extractall(directory)
Archive('../input/kkbox-music-recommendation-challenge/songs.csv.7z').extractall(directory)
Archive('../input/kkbox-music-recommendation-challenge/members.csv.7z').extractall(directory)
Archive('../input/kkbox-music-recommendation-challenge/song_extra_info.csv.7z').extractall(directory)

train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
songs = pd.read_csv('./songs.csv')
members = pd.read_csv('./members.csv')
songs_extra = pd.read_csv('./song_extra_info.csv')

### Members

In [ ]:
members.head()

### Checking for outliers in bd column

In [ ]:
sns.boxplot(x=members['bd'])

In [ ]:
plt.figure(figsize=(20,15)) 
sns.countplot(x ='bd', data = members) 
plt.xticks(rotation='90')
plt.title('Count plot for bd for members.csv',fontsize=30)
plt.show() 

In [ ]:
members["bd"].describe()

In [ ]:
count=0
for i in members["bd"]:
    if i<=50 and i>=10:
        count+=1
percentage=(count/len(members["bd"]))*100
percentage=("{:.2f}".format(percentage))
print(percentage,"% of user is between 10 and 50")

#### Along with plots when we used .describe() function we have found 50% value is zero's which means half of our data is outliers and we also found that we have max value as 1051 and min value as -43 as all we know no one can have age belonging to these number's we have found 40.74% user has age between 10-50 however if we remove outliers from our data then this percentage value will increase to 98-99% because we have 50% of our data as zero's

### Gender Column

In [ ]:
import matplotlib as mpl

mpl.rcParams['font.size'] = 20.0
labels = ['Male','Female']
plt.figure(figsize = (10, 10))
sizes = pd.value_counts(members.gender)
patches, texts, autotexts = plt.pie(sizes, 
                                    labels=labels, autopct='%.0f%%',
                                    shadow=False, radius=1,startangle=90)
for t in texts:
    t.set_size('smaller')
plt.legend()
plt.show()

In [ ]:
count = members["gender"].isna().sum()
percentage=(count/len(members['gender']))*100
percentage = "{:.3f}".format(percentage)
print("Total count of gender column of members                   :  ",len(members['gender']))
print("Number of NaN value in gender column of members           :  ",count)
print("Number of non-NaN value in gender coumn of members        :  ",(len(members['gender'])-count))
print("Total percentage of NaN value in gender column of members :  ",percentage+" %")
print("-----------------------------------------------------------------------")

### City column

In [ ]:
plt.figure(figsize=(20,15))
sns.countplot(x ='city', data = members) 
plt.xticks(rotation='90')
plt.title('Count plot for city for members.csv',fontsize=30)
plt.show() 

### Registered via column

In [ ]:
plt.figure(figsize=(15,10))
sns.countplot(x ='registered_via', data = members) 
plt.title('Count plot for registered_via for members.csv',fontsize=30)
plt.show() 

In [ ]:
members["registration_init_time"].head()

In [ ]:
members["expiration_date"].head()

### Format Conversion

In [ ]:
import pandas as pd
import numpy as np 
import datetime
members["expiration_date"]= pd.to_datetime(members["expiration_date"],format='%Y%m%d') 
members["registration_init_time"]= pd.to_datetime(members["registration_init_time"],format='%Y%m%d') 
members.dtypes

In [ ]:
members["registration_init_time"].head()
members["expiration_date"].head()

In [ ]:
plt.figure(figsize=(20,10))
plt.hist(members["registration_init_time"],bins=30,color='red')
plt.xticks(rotation='90')
plt.title('Histogram plot for registration_init for members.csv',fontsize=30)
plt.show()

### Songs_extra

In [ ]:
songs_extra.head()

In [ ]:
songs.head()

### converting song duration from seconds to minutes

In [ ]:
from tqdm import tqdm
lst=[]
for i in tqdm(songs["song_length"]):
    minutes = ((i/(1000*60))%60)
    minutes = round(minutes)
    lst.append(minutes)

In [ ]:
songs["song_duration_minutes"]=lst

In [ ]:
songs.head()


In [ ]:
plt.figure(figsize = (15, 15)) 
sns.distplot(songs.song_duration_minutes)
sns.set(font_scale=2)
plt.title('PDF for song_duration_minutes')

In [ ]:
count1=0
for i in songs["song_duration_minutes"]:
    if i<=10 and i>0:
        count1+=1
percentage1=(count1/len(songs["song_duration_minutes"]))*100
percentage1=round(percentage1)
print(percentage1,"% of song length is between 1 and 10")

In [ ]:
"""plt.figure(figsize = (15, 15)) 
sns.distplot(songs.genre_ids)
sns.set(font_scale=2)
plt.title('PDF for genre id')"""
count=0
for i in songs["genre_ids"]:
    if i=='864|857|850|843':
        count+=1
print("Total count for '864|857|850|843' in genre_ids column : ",count)

In [ ]:
lst1=[]
for i in songs["genre_ids"]:
  if i != '864|857|850|843':
    lst1.append(i)
print(lst1[:10])

In [ ]:
songs["genre_ids"]=songs["genre_ids"].astype(str)
import re
word = 'asdf*'
special_char=0
regexp = re.compile('[^0-9a-zA-Z]+')
for i in songs["genre_ids"]:
  if regexp.search(i):
    special_char+=1

print("Total count of values containing special character '|' in genre_ids column         : ",special_char)
print("Total percentage for values containing special character '|'  in genre_ids column  : ",(special_char/len(songs["genre_ids"]))*100,"%")

In [ ]:
count1 = songs["genre_ids"].isnull().sum()
print("Total count of gender column of members                   :  ",len(songs["genre_ids"]))
print("Number of NaN value in gender column of members           :  ",count1)
print("Number of non-NaN value in gender coumn of members        :  ",(len(songs["genre_ids"])-count1))
print("-----------------------------------------------------------------------")


### Merging song and song extra

In [ ]:
song_info = songs_extra.merge(songs, on='song_id', how='left')

In [ ]:
song_info.head()

In [ ]:
from collections import Counter 
import pandas as pd  
d = Counter(song_info["genre_ids"]) 
df = pd.DataFrame.from_records(d.most_common(), columns=['genre_id','count'])
print(df.head(50))

In [ ]:
df.describe()

### Nan values in all columns

In [ ]:
print("Total percentage for NaN value in song_id column        : ",(song_info["song_id"].isna().sum()/len(song_info["song_id"]))*100,"%")
print("Total percentage for NaN value in name column           : ",(song_info["name"].isna().sum()/len(song_info["name"]))*100,"%")
print("Total percentage for NaN value in isrc column           : ",(song_info["isrc"].isna().sum()/len(song_info["isrc"]))*100,"%")
print("Total percentage for NaN value in song_length column    : ",(song_info["song_length"].isna().sum()/len(song_info["song_length"]))*100,"%")
print("Total percentage for NaN value in genre_ids column      : ",(song_info["genre_ids"].isna().sum()/len(song_info["genre_ids"]))*100,"%")


print("Total percentage for NaN value in artist_name column    : ",(song_info["artist_name"].isna().sum()/len(song_info["artist_name"]))*100,"%")
print("Total percentage for NaN value in composer column       : ",(song_info["composer"].isna().sum()/len(song_info["composer"]))*100,"%")
print("Total percentage for NaN value in lyricist column       : ",(song_info["lyricist"].isna().sum()/len(song_info["lyricist"]))*100,"%")
print("Total percentage for NaN value in language column       : ",(song_info["language"].isna().sum()/len(song_info["language"]))*100,"%")

In [ ]:
msn.heatmap(song_info)

### Train.csv

In [ ]:
plt.figure(figsize=(18,12))
sns.set(font_scale=2)
sns.countplot(x='source_system_tab',hue='source_system_tab',data=train)
plt.xlabel('source system tab',fontsize=30)
plt.ylabel('count',fontsize=30)
plt.xticks(rotation='45')
plt.title('Count plot for system tab there are using',fontsize=30)

In [ ]:
print("Total percentage for NaN value in source_system_tab column  : ",(train["source_system_tab"].isna().sum()/len(train["source_system_tab"]))*100,"%")

In [ ]:
plt.figure(figsize=(20,15))
sns.set(font_scale=2)
sns.countplot(x='source_screen_name',hue='source_screen_name',data=train)
sns.set(style="darkgrid")
plt.xlabel('source screen name',fontsize=30)
plt.ylabel('count',fontsize=30)
plt.xticks(rotation='45')
plt.title('Count plot for system screen name there are using',fontsize=30)
plt.tight_layout()

In [ ]:
print("Total percentage for NaN value in source_screen_name column  : ",(train["source_screen_name"].isna().sum()/len(train["source_screen_name"]))*100,"%")

In [ ]:
plt.figure(figsize=(20,15))
sns.set(font_scale=2)
sns.countplot(x='source_type',hue='source_type',data=train)
sns.set(style="darkgrid")
plt.xlabel('source type',fontsize=30)
plt.ylabel('count',fontsize=30)
plt.xticks(rotation='45')
plt.title('Count plot for source type there are using',fontsize=30)
plt.tight_layout()

In [ ]:
print("Total percentage for NaN value in source_type column  : ",(train["source_type"].isna().sum()/len(train["source_type"]))*100,"%")

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))

# plot the cumulative histogram
n, bins, patches = ax.hist(train["target"], 10, density=True, histtype='step',
                           cumulative=True, label='Empirical')

In [ ]:
print("Total percentage for NaN value in target column  : ",(train["target"].isna().sum()/len(train["target"]))*100,"%")

In [ ]:
duplicate_values = len(train["msno"])-train["msno"].nunique()
print("Total no of duplicate user id         : ",duplicate_values)
print("Total percentage of duplicate user id : ",(duplicate_values/len(train["msno"]))*100,"%")

In [ ]:
duplicate_values1 = len(train["song_id"])-train["song_id"].nunique()
print("Total no of duplicate song id         : ",duplicate_values1)
print("Total percentage of duplicate song id : ",(duplicate_values1/len(train["song_id"]))*100,"%")

### Merging song_info and train.csv

In [ ]:
train_and_members = pd.merge(train, members, on='msno', how='left')
final_train       = pd.merge(train_and_members, song_info, on='song_id', how='left')

In [ ]:
final_train.head()

In [ ]:
plt.figure(figsize=(20,15)) 
sns.countplot(x ='language', data = final_train) 
plt.xticks(rotation='90')
plt.title('Count plot for language for final_train.csv',fontsize=30)
plt.show()

In [ ]:
msn.heatmap(final_train)

In [ ]:
test.head()

In [ ]:
print("Total percentage for NaN value in source_system_tab column   : ",(test["source_system_tab"].isna().sum()/len(test["source_system_tab"]))*100,"%")
print("Total percentage for NaN value in source_screen_name column  : ",(test["source_screen_name"].isna().sum()/len(test["source_screen_name"]))*100,"%")
print("Total percentage for NaN value in source_type column         : ",(test["source_type"].isna().sum()/len(test["source_type"]))*100,"%")
print("Total percentage for NaN value in song_id column             : ",(test["song_id"].isna().sum()/len(test["song_id"]))*100,"%")
print("Total percentage for NaN value in msno column                : ",(test["msno"].isna().sum()/len(test["msno"]))*100,"%")

### Numerical Imputation

In [ ]:
from numpy import nan
final_train["gender"].fillna("gender_not_available", inplace=True)

In [ ]:
print(final_train["gender"].isnull().sum())

In [ ]:
final_train["bd"].fillna(members["bd"].mean(), inplace=True)
print(members["bd"].isnull().sum())

In [ ]:
bd_c=0
for i in final_train["bd"]:
    if i<0 and i>80:
        bd_c+=1
print(bd_c)

In [ ]:
from numpy import nan
final_train["bd"]=final_train["bd"].mask(final_train["bd"]==0).fillna(final_train["bd"].mean())

In [ ]:
final_train["bd"]=final_train["bd"].round()

In [ ]:
plt.figure(figsize=(20,15))
# count plot on single categorical variable 
sns.countplot(x ='bd', data = final_train) 
plt.xticks(rotation='90')
plt.title('Count plot for bd for members.csv',fontsize=30)
  
# Show the plot 
plt.show() 

### '|' character problem in genre

In [ ]:
final_train["genre_ids"]=final_train["genre_ids"].astype(str)

In [ ]:
import re 
lst=[]
for i in final_train["genre_ids"]:
  res = re.findall(r"[\w']+",i) #finding values with special charcter " | "
  lst.append(res[0])

In [ ]:
final_train["genre_id"]=lst

In [ ]:
final_train = final_train.drop(columns="genre_ids") #dropping existing genre_ids column
final_train = final_train.drop(columns="song_length")

In [ ]:
final_train["genre_id"]=final_train["genre_id"].replace("nan", '0')

In [ ]:
final_train["genre_id"]=pd.to_numeric(final_train["genre_id"])

In [ ]:
import statistics

median_genre_id=int(statistics.median(final_train["genre_id"]))

In [ ]:
print(median_genre_id)

In [ ]:
final_train["genre_id"]=final_train["genre_id"].replace(0,median_genre_id )

In [ ]:
final_train["genre_id"].head(50)

In [ ]:
final_train["language"].isna().sum()

In [ ]:
import statistics
median_lang=round(statistics.median(final_train["language"]))

In [ ]:
print(median_lang)

In [ ]:
from numpy import nan
final_train["language"].fillna(median_lang, inplace=True)

In [ ]:
final_train["language"].isna().sum()

In [ ]:
final_train["name"].fillna("name_not_available", inplace=True)
final_train["isrc"].fillna("isrc_not_available", inplace=True)
final_train["artist_name"].fillna("artist_not_available", inplace=True)

final_train["source_system_tab"].fillna("data_not_available", inplace=True)
final_train["source_screen_name"].fillna("data_not_available", inplace=True)
final_train["source_type"].fillna("data_not_available", inplace=True)

In [ ]:
print("Total percentage for NaN value in name column                    : ",(final_train["name"].isna().sum()/len(final_train["name"]))*100,"%")
print("Total percentage for NaN value in isrc column                    : ",(final_train["isrc"].isna().sum()/len(final_train["isrc"]))*100,"%")
print("Total percentage for NaN value in song_length column             : ",(round((final_train["song_duration_minutes"].isna().sum()/len(final_train["song_duration_minutes"]))*100)),"%")
print("Total percentage for NaN value in artist_name column             : ",(final_train["artist_name"].isna().sum()/len(final_train["artist_name"]))*100,"%")
print("Total percentage for NaN value in source_system_tab column       : ",(final_train["source_system_tab"].isna().sum()/len(final_train["source_system_tab"]))*100,"%")
print("Total percentage for NaN value in source_screen_name column      : ",(final_train["source_screen_name"].isna().sum()/len(final_train["source_screen_name"]))*100,"%")
print("Total percentage for NaN value in source_type column             : ",(final_train["source_type"].isna().sum()/len(final_train["source_type"]))*100,"%")

In [ ]:
final_train.head(2)

In [ ]:
final_train = final_train.drop(["composer","lyricist"],axis=1)

### test.csv

In [ ]:
test_members = pd.merge(test, members, on='msno', how='left')
final_test   = pd.merge(test_members, song_info, on='song_id', how='left')

In [ ]:
final_test.head()

In [ ]:
print(final_test.isnull().sum())

In [ ]:
test['source_system_tab'].fillna('no_system_tab', inplace=True)
test['source_screen_name'].fillna('no_screen_name', inplace=True)
test['source_type'].fillna('no_source_type', inplace=True)

In [ ]:
print(test.isnull().sum())

In [ ]:
import pandas as pd
import numpy as np 
import datetime
members["expiration_date"]= pd.to_datetime(members["expiration_date"],format='%Y%m%d') 
members["registration_init_time"]= pd.to_datetime(members["registration_init_time"],format='%Y%m%d') 
members.dtypes

In [ ]:
members['registration_init_time'].head(-50)

### Getting day of the week

In [ ]:
final_train['month'] = final_train['registration_init_time'].dt.month
final_train[['registration_init_time','month']].head()

In [ ]:
final_train['day'] =final_train['registration_init_time'].dt.day
final_train[['registration_init_time','day']].head()

In [ ]:
final_train['year'] =final_train['registration_init_time'].dt.year
final_train[['registration_init_time','year']].head()

In [ ]:
final_train['year'] =final_train['registration_init_time'].dt.dayofweek
final_train[['registration_init_time','year']].head()

In [ ]:
final_train['ex_month'] = final_train['expiration_date'].dt.month
final_train[['expiration_date','ex_month']].head()

In [ ]:
final_train['ex_day'] = final_train['expiration_date'].dt.day
final_train[['expiration_date','ex_day']].head()

In [ ]:
final_train['ex_year'] = final_train['expiration_date'].dt.year
final_train[['expiration_date','ex_year']].head()

In [ ]:
final_train['ex_dayofweek'] = final_train['expiration_date'].dt.dayofweek
final_train[['expiration_date','ex_dayofweek']].head()

In [ ]:
final_train.head()

In [ ]:
from collections import Counter 
import pandas as pd  
d = Counter(final_train["name"]) 
df = pd.DataFrame.from_records(d.most_common(), columns=['song_name','count'])
print(df.head(-10))

In [ ]:
final_train["artist_name"].head()

In [ ]:
from collections import Counter 
import pandas as pd  
d = Counter(final_train["artist_name"]) 
df1 = pd.DataFrame.from_records(d.most_common(), columns=['artist_name','count'])
print(df1.head(10))

In [ ]:
from collections import Counter 
import pandas as pd  
d = Counter(final_train["song_duration_minutes"]) 
df2 = pd.DataFrame.from_records(d.most_common(), columns=['song_duration_minutes','count'])
print(df2.head(10))

In [ ]:
import statistics

median_song=round(statistics.median(final_train["song_duration_minutes"])) #finding median value 
median_song

In [ ]:
from numpy import nan
final_train["song_duration_minutes"]=final_train["song_duration_minutes"].mask(final_train["song_duration_minutes"]==0.0).fillna(median_song)

In [ ]:
final_train["song_duration_minutes"]=final_train["song_duration_minutes"].astype(int)

In [ ]:
from collections import Counter 
import pandas as pd  
d = Counter(final_train["song_duration_minutes"]) 
df3 = pd.DataFrame.from_records(d.most_common(), columns=['song_duration_minutes','count'])
print(df3.head(10))

In [ ]:
plt.figure(figsize=(20,15))
# count plot on single categorical variable 
sns.countplot(x ='song_duration_minutes', data = final_train) 
plt.xticks(rotation='90')
plt.title('Count plot for bd for members.csv',fontsize=30)
  
# Show the plot 
plt.show()

In [ ]:
isrc                             = final_train['isrc']
final_train['country_code']      = isrc.str.slice(0, 2)
final_train['registration_code'] = isrc.str.slice(2, 5)
final_train['song_year']         = isrc.str.slice(5, 7)

In [ ]:
final_train['song_year'] = final_train['song_year'] .replace("no", 0)
final_train['song_year']=final_train['song_year'].astype(int)
import statistics
median_value = int(statistics.median(final_train["song_year"])) #finding median value 
final_train['song_year'] = final_train['song_year'].apply(lambda x: 2000+x if x < 18 else 1900+x)

In [ ]:
member_song_count = final_train.groupby('msno').count()['song_id'].to_dict()
final_train['member_song_count'] = final_train['msno'].apply(lambda x: member_song_count[x])

#artist count for each song
artist_song_count = final_train.groupby('artist_name').count()['song_id'].to_dict()
final_train['artist_song_count'] = final_train['artist_name'].apply(lambda x: artist_song_count[x])

#genre count for each song
first_genre_id_song_count = final_train.groupby('genre_id').count()['song_id'].to_dict()
final_train['genre_id'] = final_train['genre_id'].apply(lambda x: first_genre_id_song_count[x])

#language count for each song
lang_song_count = final_train.groupby('language').count()['song_id'].to_dict()
final_train['lang_song_count'] = final_train['language'].apply(lambda x: lang_song_count[x])

#user count for each song
song_member_count = final_train.groupby('song_id').count()['msno'].to_dict()
final_train['song_member_count'] = final_train['song_id'].apply(lambda x: song_member_count[x])

#agecount for each song
age_song_count = final_train.groupby('bd').count()['song_id'].to_dict()
final_train['age_song_count'] = final_train['bd'].apply(lambda x: age_song_count[x])
final_train = final_train.drop(columns="isrc")
final_train.head()

In [ ]:
final_train.to_csv('./copy_final_train.csv', index=False)